# Oxylabs Search
## Overview

This notebook goes over how to use the Oxylabs search component.

First, you need to set up the proper API keys and environment variables. Create your API user credentials: Sign up for a free trial or purchase the product in the [Oxylabs dashboard](https://dashboard.oxylabs.io/en/registration) to create your API user credentials (OXYLABS_USERNAME and OXYLABS_PASSWORD).



## Instantiation
The integration lives in the `langchain-community` package.

In [ ]:
# %pip install -U langchain-community oxylabs

In [ ]:
import getpass
import os
from pprint import pprint

os.environ["OXYLABS_USERNAME"] = getpass.getpass()
os.environ["OXYLABS_PASSWORD"] = getpass.getpass()

## Invocation
#### Basic usage

In [ ]:
from langchain_community.utilities import OxylabsSearchAPIWrapper

search = OxylabsSearchAPIWrapper()

In [ ]:
pprint(search.run("Python Programming Language"))

#### Number of results
You may use the parameters `start_page`, `pages` and `limit` to specify the number of results. The default values are:  
    * `start_page = 1`  
    * `pages = 1`  
    * `limit = 5`  

In [ ]:
search = OxylabsSearchAPIWrapper(
    params={
        "start_page": 1,
        "pages": 2,
        "limit": 10
    }
)


In [ ]:
pprint(search.run("Python Programming Language"))

#### Image Content Display
You may use the `include_binary_image_data` parameter to manage the display of binary image data. By default, this parameter is set to `False`, and the image data is replaced with the text "Redacted base64 image string...". 

In [ ]:
search = OxylabsSearchAPIWrapper(
    include_binary_image_data=True,
    params={
        "start_page": 1,
        "pages": 2,
        "limit": 10
    }
)
pprint(search.run("Python Programming Language"))

#### Result Filtering / Ordering

Default result group order :  
* `knowledge_graph`  
* `combined_search_result`  
* `product_information`  
* `local_information`  
* `search_information`  
  
You may want to adjust the display order of result groups and exclude the `product_information`, `local_information`  and `combined_search_result` groups in cases where only abstract knowledge is needed. For example, this configuration allows tailored result presentation.

In [ ]:
search = OxylabsSearchAPIWrapper(
    params={
        "start_page": 1,
        "pages": 1,
        "limit": 10,
        "geo_location": "Kairo,Egypt",
        "result_categories": [
            "search_information",
            "knowledge_graph",            
        ]
    }
)
pprint(search.run("Visiting Great Pyramid of Giza in Egypt"), indent=4)


You may want to adjust the display order of result groups and exclude abstract knowledge when focusing specifically on products and local services. In the following example reorders result groups to prioritise search information and local services, omitting abstract knowledge to enhance relevance for this context.

In [ ]:
search = OxylabsSearchAPIWrapper(
    params={
        "start_page": 1,
        "pages": 5,
        "limit": 15,
        "geo_location": "Belgium",
        "result_categories": [
            "search_information",
            "local_information",            
        ]
    }
)
pprint(search.run("Open Working Space in Belgium."))

#### Raw Results

The wrapper method `results()` returns all raw results in JSON format.

In [ ]:
search = OxylabsSearchAPIWrapper()

In [ ]:
search.results("Python Programming Language")

### Invoke directly with args

In [ ]:
from langchain_community.tools.oxylabs_search import OxylabsSearchResults, OxylabsSearchRun
from langchain_community.utilities import OxylabsSearchAPIWrapper

api_wrapper = OxylabsSearchAPIWrapper()

#### Tool `OxylabsSearchRun`

In [ ]:
tool_run = OxylabsSearchRun(wrapper=api_wrapper)

In [ ]:
response_run = tool_run.invoke({
    "query": "Visit restaurants in Vilnius.",
    "geo_location": "Vilnius,Lithuania",    
})

pprint(response_run)

In [ ]:
tool_run = OxylabsSearchRun(
    wrapper=api_wrapper,
    kwargs={
        "result_categories": [
            "local_information",  
            "combined_search_result",
        ]
    }
)

In [ ]:
response_run = tool_run.invoke({
    "query": "Visit restaurants in Vilnius.",
    "geo_location": "Vilnius,Lithuania",    
})

pprint(response_run)

In [ ]:
tool_results = OxylabsSearchResults(wrapper=api_wrapper)

In [ ]:
import json

# .invoke wraps utility.results
response_results = tool_results.invoke({
    "query": "What is the weather in Shanghai?",
    "geo_location": "China",  
})
response_results = json.loads(response_results)
for item in response_results:
    print(item)

### Invoke with ToolCall

## Chaining

In [ ]:
# Install the required libraries
%pip install --upgrade --quiet langchain langchain-openai langchainhub langchain-community

In [ ]:
# Import necessary modules
import getpass
import os
from langchain import hub
from langchain.agents import AgentExecutor, create_tool_calling_agent
from langchain_openai import ChatOpenAI  # Use OpenAIChat instead of AzureChatOpenAI

# Set up OpenAI API credentials
os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter your OpenAI API key: ")

# Define assistant instructions and pull a base prompt template
instructions = """You are an assistant."""
base_prompt = hub.pull("langchain-ai/openai-functions-template")
prompt = base_prompt.partial(instructions=instructions)

# Initialize OpenAI chat model
llm = ChatOpenAI(
    openai_api_key=os.environ["OPENAI_API_KEY"]
)

# Define tools and agent setup
tool = OxylabsSearchRun(wrapper=api_wrapper)
tools = [tool]
agent = create_tool_calling_agent(llm, tools, prompt)

# Set up and invoke the agent executor
agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    verbose=True,
)
agent_executor.invoke({"input": "What happened in the latest Burning Man floods?"})

In [ ]:
agent_executor.invoke({"input": "Who won latest 2024 Lithuanian elections and why, please make an politological analysis essay from search results."})

In [ ]:
agent_executor.invoke({"input": "What is the most profitable company in lithuania in 2024?"})

## API reference